# ML Model Example

This example provides a simply query tutorial for InfluxDB Cloud Dedicated. 

In [1]:
import influxdb_client_3 as InfluxDBClient3
import pandas as pd
import numpy as np
from keras.layers import Input, Dense, LSTM, RepeatVector, TimeDistributed, BatchNormalization, Normalization
from keras.models import Model
from keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Model, load_model
from keras.layers import Input, Dense, LSTM, RepeatVector, TimeDistributed,  Conv1D, Bidirectional
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam


import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
from huggingface_hub import push_to_hub_keras
pio.renderers.default = "vscode"
import os, datetime


2023-11-02 18:36:23.264879: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/jayclifford/Documents/repos/Apache-Arrow-Tutorial/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
token = ''
host = 'eu-central-1-1.aws.cloud2.influxdata.com'    
org = 'Jay-IOx'
db = 'factory'

client = InfluxDBClient3.InfluxDBClient3(token=token,
                         host=host,
                         org=org,
                         database=db) 
    
    


In [3]:
query = '''SELECT vibration FROM machine_data WHERE "machineID" = 'machine2' AND time >= now() - 30m'''
table = client.query(query=query, language="influxql")
normal_data = table.to_pandas().drop(columns=['iox::measurement']).set_index('time')
print(normal_data)
fig = px.line(normal_data, y=["vibration"], title='raw vibration')
fig.show()

                            vibration
time                                 
2023-11-02 18:06:33.480176         87
2023-11-02 18:06:33.971890         89
2023-11-02 18:06:34.473278         90
2023-11-02 18:06:34.978106         89
2023-11-02 18:06:35.504956         89
...                               ...
2023-11-02 18:27:31.614922         87
2023-11-02 18:27:32.107534         85
2023-11-02 18:27:32.610040         86
2023-11-02 18:27:33.108254         87
2023-11-02 18:27:33.610810         87

[1899 rows x 1 columns]


In [4]:
query = '''SELECT vibration FROM machine_data WHERE "machineID" = 'machine1' AND time >= now() - 30m'''
table = client.query(query=query, language="influxql")
anom_data = table.to_pandas().drop(columns=['iox::measurement']).set_index('time')
fig2 = px.line(anom_data, y=["vibration"], title='raw vibration')
fig2.show()

In [5]:
from keras.layers import Layer
import tensorflow as tf

class SequenceLayer(Layer):
    def __init__(self, timesteps, **kwargs):
        self.timesteps = timesteps
        super(SequenceLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        super(SequenceLayer, self).build(input_shape)

    def call(self, x):
        sequences = []
        for i in range(self.timesteps):
            sequences.append(x[i:-(self.timesteps-i-1)])
        return tf.stack(sequences, axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.timesteps, input_shape[1])

# Preprocess the data
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(normal_data)

# Build the autoencoder model
input_dim = df_scaled.shape[1]  # number of features
timesteps = 35  # you can change this based on your sequence length

inputs = Input(shape=(input_dim,))
sequences = SequenceLayer(timesteps)(inputs)
inputs = Input(shape=(timesteps, input_dim))
encoded = LSTM(16, activation='relu', return_sequences=True)(inputs)
encoded = LSTM(4, activation='relu', return_sequences=False)(encoded)
decoded = RepeatVector(timesteps)(encoded)
decoded = LSTM(4, activation='relu', return_sequences=True)(decoded)
decoded = LSTM(16, activation='relu', return_sequences=True)(decoded)
decoded = TimeDistributed(Dense(input_dim))(decoded)

autoencoder = Model(inputs, decoded)

autoencoder.compile(optimizer='adam', loss='mse')

# Train the model
X_train, X_test = train_test_split(df_scaled, test_size=0.2, random_state=42)

# Directory to store logs
log_dir = os.path.join(
    "logs",
    "fit",
    datetime.datetime.now().strftime("%Y%m%d-%H%M%S"),
)

# Creating the TensorBoard callback
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

history = autoencoder.fit(X_train, X_train, epochs=100, batch_size=16, validation_data=(X_test, X_test), verbose=1, callbacks=[tensorboard_callback])


Epoch 1/100
95/95 [==============================] - 7s 24ms/step - loss: 0.1891 - val_loss: 0.1326
Epoch 2/100
95/95 [==============================] - 2s 17ms/step - loss: 0.1287 - val_loss: 0.1246
Epoch 3/100
95/95 [==============================] - 1s 15ms/step - loss: 0.1241 - val_loss: 0.1218
Epoch 4/100
95/95 [==============================] - 1s 15ms/step - loss: 0.1218 - val_loss: 0.1201
Epoch 5/100
95/95 [==============================] - 1s 15ms/step - loss: 0.1204 - val_loss: 0.1194
Epoch 6/100
95/95 [==============================] - 1s 15ms/step - loss: 0.1200 - val_loss: 0.1178
Epoch 7/100
95/95 [==============================] - 1s 15ms/step - loss: 0.1189 - val_loss: 0.1170
Epoch 8/100
95/95 [==============================] - 1s 16ms/step - loss: 0.1187 - val_loss: 0.1167
Epoch 9/100
95/95 [==============================] - 1s 16ms/step - loss: 0.1184 - val_loss: 0.1169
Epoch 10/100
95/95 [==============================] - 1s 15ms/step - loss: 0.1175 - val_loss: 0.1163

In [ ]:

 # Use the Autoencoder to predict on the anomalous data
predictions = autoencoder.predict(anom_data)

# Calculate reconstruction error for each sequence
mse = np.mean(np.power(anom_data.values[-predictions.shape[0]:] - predictions[:, -1, :], 2), axis=1)

# Scale the MSE to a percentage
min_mse = np.min(mse)
max_mse = np.max(mse)
mse_percentage = ((mse - min_mse) / (max_mse - min_mse)) * 100

# Detect anomalies by comparing the scaled reconstruction error to some threshold
threshold = 82  # Define a threshold value (in percentage)

# Add 'is_anomalous' column to the DataFrame
anom_data = anom_data.iloc[-predictions.shape[0]:].copy()
anom_data['is_anomalous'] = mse_percentage > threshold
anom_data['mse_percentage'] = mse_percentage


# Create a new figure
fig = go.Figure()


# Add a line for the actual load
fig.add_trace(go.Scatter(
    x=anom_data.index,
    y=anom_data["vibration"],
    mode='lines',
    name='vibration',
    line=dict(color='green'),  # specify line color here
))

# Add a line for the anomalous load
fig.add_trace(go.Scatter(
    x=anom_data[anom_data['is_anomalous']].index,
    y=anom_data[anom_data['is_anomalous']]["vibration"],
    mode='markers',
    name='Anomalous Vibration',
    marker=dict(color='red', size=5),  # specify marker color and size here
))

# Show the plot
fig.show()
anom_data.to_csv('anomalous_data_result.csv')

In [ ]:

 # Use the Autoencoder to predict on the anomalous data
predictions = autoencoder.predict(normal_data)

# Calculate reconstruction error for each sequence
mse = np.mean(np.power(normal_data.values[-predictions.shape[0]:] - predictions[:, -1, :], 2), axis=1)

# Scale the MSE to a percentage
min_mse = np.min(mse)
max_mse = np.max(mse)
mse_percentage = ((mse - min_mse) / (max_mse - min_mse)) * 100

# Detect anomalies by comparing the scaled reconstruction error to some threshold
threshold = 82  # Define a threshold value (in percentage)

# Add 'is_anomalous' column to the DataFrame
normal_data = normal_data.iloc[-predictions.shape[0]:].copy()
normal_data['is_anomalous'] = mse_percentage > threshold
normal_data['mse_percentage'] = mse_percentage


# Create a new figure
fig = go.Figure()


# Add a line for the actual load
fig.add_trace(go.Scatter(
    x=normal_data.index,
    y=normal_data["vibration"],
    mode='lines',
    name='vibration',
    line=dict(color='green'),  # specify line color here
))

# Add a line for the anomalous load
fig.add_trace(go.Scatter(
    x=normal_data[normal_data['is_anomalous']].index,
    y=normal_data[normal_data['is_anomalous']]["vibration"],
    mode='markers',
    name='Anomalous Vibration',
    marker=dict(color='red', size=5),  # specify marker color and size here
))

# Show the plot
fig.show()
normal_data.to_csv('normal_data_result.csv')

In [ ]:
query = '''SELECT vibration FROM machine_data WHERE "machineID" = 'machine3' AND time >= now() - 1m'''
table = client.query(query=query, language="influxql")
test_data = table.to_pandas().drop(columns=['iox::measurement']).set_index('time')


 # Use the Autoencoder to predict on the anomalous data
predictions = autoencoder.predict(test_data)

# Calculate reconstruction error for each sequence
mse = np.mean(np.power(test_data.values[-predictions.shape[0]:] - predictions[:, -1, :], 2), axis=1)

# Scale the MSE to a percentage
min_mse = np.min(mse)
max_mse = np.max(mse)
mse_percentage = ((mse - min_mse) / (max_mse - min_mse)) * 100

# Detect anomalies by comparing the scaled reconstruction error to some threshold
threshold = 83  # Define a threshold value (in percentage)

# Add 'is_anomalous' column to the DataFrame
test_data = test_data.iloc[-predictions.shape[0]:].copy()
test_data['is_anomalous'] = mse_percentage > threshold
test_data['mse_percentage'] = mse_percentage


# Create a new figure
fig = go.Figure()


# Add a line for the actual load
fig.add_trace(go.Scatter(
    x=test_data.index,
    y=test_data["vibration"],
    mode='lines',
    name='vibration',
    line=dict(color='green'),  # specify line color here
))

# Add a line for the anomalous load
fig.add_trace(go.Scatter(
    x=test_data[test_data['is_anomalous']].index,
    y=test_data[test_data['is_anomalous']]["vibration"],
    mode='markers',
    name='Anomalous Vibration',
    marker=dict(color='red', size=5),  # specify marker color and size here
))

# Show the plot
fig.show()
test_data.to_csv('test_data_result.csv')



In [ ]:

push_to_hub_keras(model = autoencoder,
    repo_id = "jayclifford345/vibration-autoencoder",
    tags = ["anomaly-detection", "vibration", "autoencoder"],
    plot_model=True,
    log_dir=log_dir,
    token='',
    include_optimizer=True,
    config=autoencoder.get_config()
)